In [100]:
import json
import pickle

In [14]:
def compress_1_number_vbe(number):
    binary = bin(number)[2:]
    res = ''
    i = 0
    while len(binary)>7:
        res = '0'+binary[len(binary)-7:]+res
        binary = binary[:len(binary)-7]
        i+=1
    res ='1'+ binary.zfill(7) + res
    i+=1
    return int(res,base=2),i

In [16]:
def compress_vbe(numbers):
    res = 0
    for number in numbers:
        compressed = compress_1_number_vbe(number)
        res = (res<<(compressed[1]*8))+compressed[0]
    return res

In [17]:
bin(compress_vbe([0b11111111,0b11111111]))

'0b10000001011111111000000101111111'

In [65]:
def decompress_vbe(compressed_int):
    res = []
    compressed_str = bin(compressed_int)[2:]
    while len(compressed_str)>0:
        temp = ''
        while len(compressed_str)>0 and compressed_str[len(compressed_str)-8]=='0':
            temp = compressed_str[len(compressed_str)-7:]+temp
            compressed_str = compressed_str[:len(compressed_str)-8]
        temp = compressed_str[len(compressed_str)-7:]+temp
        compressed_str = compressed_str[:len(compressed_str)-8]
        res = [int(temp,base=2)]+res
    return res

In [33]:
def compress_gap(numbers):
    res = [numbers[0]]
    for index in range(1,len(numbers)):
        res.append(numbers[index]-numbers[index-1])
    return res

In [34]:
def decompress_gap(compressed):
    res = [compressed[0]]
    for index in range(1,len(compressed)):
        res.append(compressed[index]+res[index-1])
    return res

In [35]:
def compress_dual(numbers):
    return compress_vbe(compress_gap(numbers))

In [36]:
def decompress_dual(compressed):
    return decompress_gap(decompress_vbe(compressed))

In [40]:
with open('index_one.json') as f:
  data = json.load(f)

In [41]:
def string_to_nbr(doc_id):
    result = 0
    for char in doc_id:
        result = result<<8
        result+=ord(char)
    return result

In [42]:
def nbr_to_string(nbr):
    result = ''
    while nbr>0:
        temp = nbr%256
        nbr = nbr>>8
        result = chr(temp)+result
    return result

In [57]:
def compress(dictionary):
    result = []
    last_key = 0
    int_keys = []
    for key in dictionary.keys():
        int_keys.append(string_to_nbr(key))
    for key in sorted(int_keys):
        #print(key,string_to_nbr(key),string_to_nbr(key)>last_key)
        result.append(key-last_key)
        last_key = key
        key = nbr_to_string(key)
        result.append(len(dictionary[key]))
        result=result+compress_gap(dictionary[key])
    #return result
    return compress_vbe(result)

In [73]:
def decompress(vbe_compressed):
    result = {}
    compressed_list = decompress_vbe(vbe_compressed)
    int_key = 0
    while len(compressed_list)>0:
        temp = compressed_list.pop(0)
        int_key = int_key + temp
        key = nbr_to_string(int_key)
        length = compressed_list.pop(0)
        result[key] = decompress_gap(compressed_list[:length])
        compressed_list = compressed_list[length:]
    return result

In [81]:
def compress_all(dictionary):
    res = {}
    for key in dictionary:
        res[key] = compress(dictionary[key])
    return res

In [83]:
compressed = compress_all(data)

In [85]:
with open('out.json', 'w') as outfile:
    json.dump(compressed, outfile)

In [93]:
with open('out_bin','wb') as outfile:
    pickle.dump(compressed,outfile)

In [95]:
with open('out_bin','rb') as infile:
    saved = pickle.load(infile)

In [98]:
def decompress_all(compressed):
    res = {}
    for key in compressed:
        res[key] = decompress(compressed[key])
    return res

In [ ]:
decompress_all(saved)